<a href="https://colab.research.google.com/github/layan121213/AASQ/blob/main/L.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pprint
from gensim.models import word2vec , KeyedVectors
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import nltk 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('stopwords')
nltk.download('punkt')
from textblob import TextBlob
import gensim
from nltk.stem.lancaster import LancasterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from F1 import evaluationMetric

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/Colab Notebooks/GoogleNews-vectors-negative300.bin" , binary=True)

In [6]:
pip install gingerit

     |████████████████████████████████| 63 kB 1.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
#import cloudscraper
# session = cloudscraper.create_scraper()

In [ ]:
pip install cloudscraper

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

lancaster = LancasterStemmer()


from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
stemmer = PorterStemmer()

def stem_words(text):
    word_tokens = word_tokenize(text)
    stems = [stemmer.stem(word) for word in word_tokens]
    return ' '.join(stems)

def stemming(text):
    
    snowball = SnowballStemmer(language='english')
    
    list=[]
    for token in word_tokenize(text):
        list.append(snowball.stem(token))
    
    return ' '.join(list)

def preprocessing(t):
    # print(word_tokenize(t))
    t = str(t).lower()  # lowercasing
    no_punc = t.translate(str.maketrans(
        '', '', string.punctuation))  # punc removal
    # stem = stem_words(no_punc)
    # print("stem",stem)
    no_stopWords = stopwords.words('english')  # stop words removal
    text_tokens = word_tokenize(no_punc)
    not_stopword = [word for word in text_tokens if not word in no_stopWords]
    # print("not_stopword",not_stopword)
    return not_stopword


def clean(text):
    cleaned = ' '.join([str(i) for i in text]).replace('[\'', ' ').replace('\']', '')
    return cleaned


def listToString(s): 
    str1 = "" 
    for ele in s: 
        str1 += ele  
    return str1 

w2v_vocab = set(model.vocab)
index2word_set = set(model.wv.index2word)

def avg_feature_vector(sentence, model, num_features, index2word_set):
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

In [ ]:
from scipy import spatial
# from spellchecker import SpellChecker
import math
# from autocorrect import Speller
from scipy.sparse import csr_matrix
from numpy import count_nonzero
import cloudscraper
from gingerit.gingerit import GingerIt

parser = GingerIt()
# check = Speller(lang='en')
# spell = SpellChecker()
# check = Speller(lang='en')
text = "An algoriithm is a sequence of steps or a plan to complete an action or do something."
ModwlA= parser.parse(text)['result']
toString2 = clean(preprocessing(ModwlA))
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/test.xlsx", usecols="A", index_col=None)
a = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/test.xlsx", usecols="B", index_col=None)
count =0
jacc = 0
cos = 0.0
div = 0
list = []  
val = []


for rowIndex, row in df.iterrows(): 
    for value in row.items():
        # print("res>>>>>",value[1])
        res=parser.parse(value[1])['result']
        # print("gingerit:",res)
        toString = clean(preprocessing(res))
        # print("Pre>>>>>",toString)
        s1_afv=avg_feature_vector(toString2, model=model, num_features=300, index2word_set=index2word_set)
        S = csr_matrix(s1_afv)
        B = S.todense()
        s2_afv = avg_feature_vector(toString, model=model, num_features=300, index2word_set=index2word_set)
        L = csr_matrix(s2_afv)
        G = L.todense()
        sim = 1 - spatial.distance.cosine(B, G)
        if( math.isnan(sim)):
          print(toString)
          sim=0.0
          print(sim)
          val.append(sim)
        else:
          print(toString)
          print(sim)
          val.append(sim)

        # cos += sim
        # print(toString)
        # print(sim)
        # print("cos",count)
        
predicted = [float(x) for x in val]
actual = [int(x) for x in a.values] 
f1 = evaluationMetric(actual,predicted)
print(f1)
# print(index2word_set)
# sparsity = 1.0 - count_nonzero(s1_afv)/ s1_afv.size
# print("s1_afv", sparsity)
# sparsity2 = 1.0 - count_nonzero(s2_afv)/ s2_afv.size
# print("s2_afv", sparsity2)
# print("\njac average: "+str(jacc/count))
# print("average: "+str(cos/count))
# print("div average: "+str(div/count))